In [1]:
pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 6.2 MB 4.4 MB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 281 kB 65.9 MB/s 
     |████████████████████████████████| 103 kB 44.5 MB/s 
     |████████████████████████████████| 36.8 MB 1.2 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 109 kB 42.3 MB/s 
     |████████████████████████████████| 78 kB 6.3 MB/s 
     |████████████████████████████████| 51 kB 606 kB/s 
     |████████████████████████████████| 41 kB 402 kB/s 
     |████████████████████████████████| 3.2 MB 42.4 MB/s 
     |████████████████████████████████| 546 kB 49.2 MB/s 
     |████████████████████████████████| 121 kB 53.9 MB/s 
     |████████████████████████████████| 84 kB 3.1 MB/s 
     |████████████████████████████████| 90 kB 8

In [2]:
args = {'data':'cora',
        'h_feats':16,
        'lr':0.01,
        'epochs':100}

In [3]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn import GraphConv
import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('--data', type=str, default = "cora", help = "benchmark dataset")
# parser.add_argument('--h_feats', type=int, default = 16, help = "number of hidden channels")
# parser.add_argument('--lr', type=float, default = 0.01, help = "learning rate")
# parser.add_argument('--epochs', type=int, default = 100, help = "epochs")
# args = parser.parse_args()

data = str(args['data'])
h_feats = int(args['h_feats'])
lr = float(args['lr'])
epochs = int(args['epochs'])

if data == "cora":
    dataset = dgl.data.CoraGraphDataset()
elif data == "citeseer":
    dataset = dgl.data.CiteseerGraphDataset()
elif data == "pubmed":
    dataset = dgl.data.PubmedGraphDataset()

print('Number of categories:', dataset.num_classes)

g = dataset[0]


class GCN(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GraphConv(in_feats, h_feats)
        self.conv2 = GraphConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h


def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(epochs):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if e % (epochs/20) == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = GCN(g.ndata['feat'].shape[1], h_feats, dataset.num_classes)
train(g, model)

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)
Extracting file to /root/.dgl/cora_v2
Finished data loading and preprocessing.
  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.
Number of categories: 7


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 1.947, val acc: 0.144 (best 0.144), test acc: 0.142 (best 0.142)
In epoch 5, loss: 1.897, val acc: 0.646 (best 0.646), test acc: 0.682 (best 0.682)
In epoch 10, loss: 1.820, val acc: 0.690 (best 0.692), test acc: 0.699 (best 0.707)
In epoch 15, loss: 1.717, val acc: 0.688 (best 0.692), test acc: 0.696 (best 0.707)
In epoch 20, loss: 1.587, val acc: 0.690 (best 0.698), test acc: 0.689 (best 0.694)
In epoch 25, loss: 1.436, val acc: 0.722 (best 0.722), test acc: 0.712 (best 0.712)
In epoch 30, loss: 1.266, val acc: 0.714 (best 0.724), test acc: 0.722 (best 0.714)
In epoch 35, loss: 1.088, val acc: 0.732 (best 0.732), test acc: 0.732 (best 0.732)
In epoch 40, loss: 0.913, val acc: 0.742 (best 0.742), test acc: 0.744 (best 0.744)
In epoch 45, loss: 0.750, val acc: 0.760 (best 0.760), test acc: 0.751 (best 0.751)
In epoch 50, loss: 0.607, val acc: 0.762 (best 0.762), test acc: 0.760 (best 0.754)
In epoch 55, loss: 0.487, val acc: 0.770 (best 0.770), test acc: 0.766 (best 0